In [1]:
import pandas as pd

df = pd.DataFrame()

In [2]:
import os

In [3]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

45

In [4]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import re

In [5]:
for a in tqdm(range(len(list_file)), desc='Progress', ncols=77):
    with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
        resp = file.read()
        
    # = = = = = = = = = = = = = = =
    
    soup = BeautifulSoup(resp, 'lxml')
    html = etree.HTML(str(soup))
    
    # = = = = = = = = = = = = = = =
    
    title = html.xpath('//h1[@class="product__title-link"]/text()')[0].strip()

    # = = = = = = = = = = = = = = =

    subtitle = html.xpath('//h1[@class="product__title-link"]/span[@class="product__subtitle"]/text()')
    if len(subtitle) == 0:
        subtitle = ''
    else:
        subtitle = subtitle[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    artkl = re.sub('\n +', '\n', html.xpath('//div[@class="product__artkl"]/text()')[0].strip())

    # = = = = = = = = = = = = = = =
    
    product_id = html.xpath('//div[@class="product"]/@data-article-id')[0].strip()

    # = = = = = = = = = = = = = = =
    
    price = html.xpath('//div[@class="product__new-price"]/text()')[0].strip()

    # = = = = = = = = = = = = = = =
    
    list_oe = []
    list_li = html.xpath('//div[@class="product-oem__list"]/ul/li')
    for li in list_li:
        if len(li.xpath('./a')) == 0:
            list_oe.append(re.sub('^.*?OE-', '', li.xpath('./text()')[0].strip()))
        else:
            list_oe.append(re.sub('^.*?OE-', '', li.xpath('./a/text()')[0].strip()))
            
    # = = = = = = = = = = = = = = =
    
    kit = '\n'.join([re.sub('\n +', ' ', kit.strip()) for kit in html.xpath('//div[@class="product__kit"]/ul/li/text()')])

    # = = = = = = = = = = = = = = =
    
    src = html.xpath('//div[contains(@class, "main-image")]/img/@src')[0].strip()

    # = = = = = = = = = = = = = = =
    
    maker_id = ';'.join([maker.strip() for maker in html.xpath('//div[@class="compatibility__maker-title"]/@data-maker-id')])

    # = = = = = = = = = = = = = = =
    
    list_tr = html.xpath('//table[@class="product__table"]/tr')

    # = = = = = = = = = = = = = = =
    
    list_li = html.xpath('//div[@class="product-analogs__wrapper"]/ul/li')

    # = = = = = = = = = = = = = = =
    
    df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                             'Title': title,
                             'Subtitle': subtitle,
                             'Artkl': artkl,
                             'Product_Id': product_id,
                             'Price': price,
                             'Vehicle_1': '',
                             'Vehicle_2': '',
                             'OE': ';'.join(list_oe),
                             'Kit': kit,
                             'Pic': '',
                             'Url': '',
                             'Src': src,
                             'Maker_Id': maker_id}])

    for tr in list_tr:
        name = tr.xpath('./td[1]/text()')[0].strip()
        if name[-1] == ':':
            df_temp.loc[0, name.upper()[:-1]] = tr.xpath('./td[2]/text()')[0].strip()
        else:
            df_temp.loc[0, name.upper()] = tr.xpath('./td[2]/text()')[0].strip()

    for li in list_li:
        name = li.xpath('./span/text()')[0].strip()
        if name[-1] == ':':
            df_temp.loc[0, name.lower()[:-1]] = ''.join(li.xpath('./text()')).strip()
        else:
            df_temp.loc[0, name.lower()] = ''.join(li.xpath('./text()')).strip()

    df = pd.concat([df, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =
    
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|██████████████████████████████| 45/45 [00:02<00:00, 22.44it/s]


,序号,Title,Subtitle,Artkl,Product_Id,Price,Vehicle_1,Vehicle_2,OE,Kit,...,fersa bearings,meyle,timken,dt spare parts,skf,zf,FELGE LOCHZAHL,FLANSCH-Ø [MM],bta,st-templin
0,1,SKF VKBA 944 Radlagersatz,"mit Wellendichtring, 39,9 mm",Art. Nr: VKBA 944\nHersteller: SKF,1363314,"12,72 €",,,11600.21408.00;11600.21413.03;11600.21413.06;5...,"1 x Sortiment, Befestigungselemente (Art. Nr: ...",...,,,,,,,,,,
1,2,SKF VKBA 1927 Radlagersatz,40 mm,Art. Nr: VKBA 1927\nHersteller: SKF,1362409,"13,73 €",,,07 11 9 981 213;90043-63010;90043-63012;90043-...,1 x Dichtung (Art. Nr: SKF04937)\n1 x Splint (...,...,,,,,,,,,,
2,3,SKF VKBA 1931 Radlagersatz,47 mm,Art. Nr: VKBA 1931\nHersteller: SKF,1362413,"13,96 €",,,09262A-20076;09262A-25074;90043-63079;9060-200...,1 x Splint (Art. Nr: SKF01997)\n1 x Lager (Art...,...,,,,,,,,,,
3,4,SKF VKBA 530 Radlagersatz,"mit Wellendichtring, 39,9 mm",Art. Nr: VKBA 530\nHersteller: SKF,1362925,"14,24 €",,,11600.21408.00;11600.21413.03;11600.21413.06;5...,1 x Lager (Art. Nr: SKF02676)\n1 x Lager (Art....,...,,,,,,,,,,
4,5,SKF VKHB 2270 Radlager,"19x45,2x15,5 mm",Art. Nr: VKHB 2270\nHersteller: SKF,16143,"14,34 €",,,11900.21408.01;60523333;31 21 2 634 103;409610...,,...,LM 11949/LM 11910,,LM11949/LM11910,,,,,,,
5,6,SKF VKBA 542 Radlagersatz,"mit Wellendichtring, 50,3 mm",Art. Nr: VKBA 542\nHersteller: SKF,1362932,"14,48 €",,,11600.21413.03;11600.21413.06;11900.21408.01;5...,1 x Verschluss-/Schutzkappe (Art. Nr: SKF03379...,...,,,,,,,,,,
6,7,SKF VKHB 2189 Radlager,25x52x16 mm,Art. Nr: VKHB 2189\nHersteller: SKF,15809,"14,68 €",,,59130205;02.6401.25.00;7703090098;30205 J2/Q,,...,30205 F,,30205,,,,,,,
7,8,SKF VKHB 2271 Radlager,"22x45,2x15,5 mm",Art. Nr: VKHB 2271\nHersteller: SKF,16144,"14,72 €",,,31 21 1 106 032;000 980 93 02;002 980 65 02;A ...,,...,LM 12749/LM 12710,,LM12749/LM12710,,,,,,,
8,9,SKF VKHB 2237 Radlager,"35x59,1x15,9 mm",Art. Nr: VKHB 2237\nHersteller: SKF,15842,"14,72 €",,,31 21 1 106 033;31 21 1 119 571;C45709;GHB 208...,,...,L 68149/L 68110,,L68149/L68110,,,,,,,
9,10,SKF VKBA 3255 Radlagersatz,"mit Wellendichtring, 39,9 mm",Art. Nr: VKBA 3255\nHersteller: SKF,1362479,"14,72 €",,,11600.21408.00;11600.21413.03;11600.21413.06;5...,1 x Splint (Art. Nr: SKF01928)\n1 x Lager (Art...,...,,,,,,,,,,
